## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Albany,US,42.6001,-73.9662,33.01,69,100,3.11,overcast clouds
1,1,Shache,CN,38.4167,77.2406,32.59,60,0,4.68,clear sky
2,2,Contamana,PE,-7.3333,-75.0167,76.30,82,100,4.74,overcast clouds
3,3,Busselton,AU,-33.6500,115.3333,71.01,44,89,4.00,overcast clouds
4,4,Esperance,AU,-33.8667,121.9000,68.00,42,99,14.97,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Contamana,PE,-7.3333,-75.0167,76.30,82,100,4.74,overcast clouds
11,11,Mataura,NZ,-46.1927,168.8643,77.00,47,4,5.91,clear sky
12,12,Atuona,PF,-9.8000,-139.0333,77.81,76,40,12.97,scattered clouds
13,13,Monrovia,LR,6.3005,-10.7969,78.08,83,44,2.95,scattered clouds
19,19,Hilo,US,19.7297,-155.0900,80.60,61,20,3.44,few clouds
20,20,Vaini,TO,-21.2000,-175.2000,86.00,79,40,8.05,scattered clouds
23,23,Morondava,MG,-20.2833,44.2833,82.40,77,87,0.92,light rain
24,24,Rikitea,PF,-23.1203,-134.9692,77.92,77,71,11.61,broken clouds
37,37,Flinders,AU,-34.5833,150.8552,90.00,61,18,4.00,few clouds
38,38,Kavieng,PG,-2.5744,150.7967,85.28,69,82,13.69,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                151
City                   151
Country                151
Lat                    151
Lng                    151
Max Temp               151
Humidity               151
Cloudiness             151
Wind Speed             151
Current Description    151
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Contamana,PE,76.30,overcast clouds,-7.3333,-75.0167,
11,Mataura,NZ,77.00,clear sky,-46.1927,168.8643,
12,Atuona,PF,77.81,scattered clouds,-9.8000,-139.0333,
13,Monrovia,LR,78.08,scattered clouds,6.3005,-10.7969,
19,Hilo,US,80.60,few clouds,19.7297,-155.0900,
20,Vaini,TO,86.00,scattered clouds,-21.2000,-175.2000,
23,Morondava,MG,82.40,light rain,-20.2833,44.2833,
24,Rikitea,PF,77.92,broken clouds,-23.1203,-134.9692,
37,Flinders,AU,90.00,few clouds,-34.5833,150.8552,
38,Kavieng,PG,85.28,broken clouds,-2.5744,150.7967,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# Verify hotel_df has hotel names
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Contamana,PE,76.30,overcast clouds,-7.3333,-75.0167,Shiringal Touristic Park
11,Mataura,NZ,77.00,clear sky,-46.1927,168.8643,Ellie's Villa
12,Atuona,PF,77.81,scattered clouds,-9.8000,-139.0333,Villa Enata
13,Monrovia,LR,78.08,scattered clouds,6.3005,-10.7969,Sunset Inn
19,Hilo,US,80.60,few clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
20,Vaini,TO,86.00,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
23,Morondava,MG,82.40,light rain,-20.2833,44.2833,La Case Bambou
24,Rikitea,PF,77.92,broken clouds,-23.1203,-134.9692,Pension Maro'i
37,Flinders,AU,90.00,few clouds,-34.5833,150.8552,Shellharbour Resort & Conference Centre
38,Kavieng,PG,85.28,broken clouds,-2.5744,150.7967,Nusa Island Retreat


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [11]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))